## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
file_to_load = os.path.join("..", "Weather_Database", "WeatherPy_Database.csv")
city_data_df = pd.read_csv(file_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hithadhoo,MV,-0.6000,73.0833,83.88,69,95,11.77,overcast clouds
1,1,Auki,SB,-8.7676,160.7034,79.41,83,100,6.31,overcast clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,41.11,82,0,14.09,clear sky
3,3,Dekar,BW,-21.5333,21.9333,55.08,54,0,8.77,clear sky
4,4,Hermanus,ZA,-34.4187,19.2345,57.79,50,67,7.99,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hithadhoo,MV,-0.6000,73.0833,83.88,69,95,11.77,overcast clouds
1,1,Auki,SB,-8.7676,160.7034,79.41,83,100,6.31,overcast clouds
6,6,Rikitea,PF,-23.1203,-134.9692,74.53,68,61,2.89,broken clouds
7,7,Ca Mau,VN,9.1769,105.1500,80.55,81,99,12.75,overcast clouds
8,8,Manado,ID,1.4870,124.8455,81.37,69,20,4.16,few clouds
9,9,San Patricio,US,28.0170,-97.5169,82.87,75,68,12.08,broken clouds
10,10,Jenison,US,42.9072,-85.7920,70.70,82,100,1.99,overcast clouds
12,12,Lang Son,VN,21.8333,106.7333,75.00,89,95,6.33,moderate rain
17,17,Kapaa,US,22.0752,-159.3190,81.48,68,40,3.00,scattered clouds
18,18,Pangai,TO,-19.8000,-174.3500,77.88,82,96,6.53,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                298
City                   298
Country                296
Lat                    298
Lng                    298
Max Temp               298
Humidity               298
Cloudiness             298
Wind Speed             298
Current Description    298
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                296
City                   296
Country                296
Lat                    296
Lng                    296
Max Temp               296
Humidity               296
Cloudiness             296
Wind Speed             296
Current Description    296
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hithadhoo,MV,83.88,overcast clouds,-0.6000,73.0833,
1,Auki,SB,79.41,overcast clouds,-8.7676,160.7034,
6,Rikitea,PF,74.53,broken clouds,-23.1203,-134.9692,
7,Ca Mau,VN,80.55,overcast clouds,9.1769,105.1500,
8,Manado,ID,81.37,few clouds,1.4870,124.8455,
9,San Patricio,US,82.87,broken clouds,28.0170,-97.5169,
10,Jenison,US,70.70,overcast clouds,42.9072,-85.7920,
12,Lang Son,VN,75.00,moderate rain,21.8333,106.7333,
17,Kapaa,US,81.48,scattered clouds,22.0752,-159.3190,
18,Pangai,TO,77.88,overcast clouds,-19.8000,-174.3500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    print(f"Retrieving Results for Index {index}: {lat, lng}.")
   
    hotels = requests.get(base_url, params=params).json()
 
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    #try:
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Retrieving Results for Index 0: (-0.6, 73.0833).
Retrieving Results for Index 1: (-8.7676, 160.7034).
Retrieving Results for Index 6: (-23.1203, -134.9692).
Retrieving Results for Index 7: (9.1769, 105.15).
Retrieving Results for Index 8: (1.4869999999999999, 124.8455).
Retrieving Results for Index 9: (28.017, -97.5169).
Retrieving Results for Index 10: (42.9072, -85.792).
Retrieving Results for Index 12: (21.8333, 106.7333).
Retrieving Results for Index 17: (22.0752, -159.319).
Retrieving Results for Index 18: (-19.8, -174.35).
Retrieving Results for Index 20: (-3.6667, 152.4333).
Retrieving Results for Index 21: (7.6244, -11.8332).
Retrieving Results for Index 22: (39.1834, -84.5333).
Retrieving Results for Index 24: (-21.2078, -159.775).
Retrieving Results for Index 27: (10.9995, 122.6711).
Retrieving Results for Index 28: (-0.7393, -90.3518).
Retrieving Results for Index 31: (-1.9417, -54.7383).
Retrieving Results for Index 37: (14.8961, -24.4956).
Retrieving Results for Index 43: 

Retrieving Results for Index 354: (18.4167, 75.2).
Retrieving Results for Index 357: (17.6667, -96.1333).
Retrieving Results for Index 358: (37.3482, 59.6143).
Hotel not found... skipping.
Retrieving Results for Index 365: (27.0377, 14.4283).
Retrieving Results for Index 368: (22.9333, 57.5333).
Retrieving Results for Index 372: (11.2833, -15.8333).
Retrieving Results for Index 377: (-15.9169, -46.1056).
Retrieving Results for Index 378: (14.0, -88.35).
Retrieving Results for Index 383: (11.5, 42.5).
Retrieving Results for Index 385: (-15.3972, -42.8664).
Retrieving Results for Index 386: (4.9533, -77.366).
Retrieving Results for Index 387: (24.4167, 76.5667).
Retrieving Results for Index 392: (13.6017, 26.6876).
Hotel not found... skipping.
Retrieving Results for Index 393: (34.9833, 139.8667).
Retrieving Results for Index 394: (18.7369, 7.3853).
Retrieving Results for Index 399: (22.05, -98.1667).
Retrieving Results for Index 400: (35.1717, -103.725).
Retrieving Results for Index 401

Retrieving Results for Index 691: (6.4635, -2.3194).


In [9]:
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hithadhoo,MV,83.88,overcast clouds,-0.6000,73.0833,Scoop Guest House
1,Auki,SB,79.41,overcast clouds,-8.7676,160.7034,Auki Motel
6,Rikitea,PF,74.53,broken clouds,-23.1203,-134.9692,Pension Maro'i
7,Ca Mau,VN,80.55,overcast clouds,9.1769,105.1500,Khách Sạn Mường Thanh Luxury Cà Mau
8,Manado,ID,81.37,few clouds,1.4870,124.8455,Aryaduta Hotel
9,San Patricio,US,82.87,broken clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"
10,Jenison,US,70.70,overcast clouds,42.9072,-85.7920,"Holiday Inn Express Grand Rapids SW, an IHG Hotel"
12,Lang Son,VN,75.00,moderate rain,21.8333,106.7333,Khai Nga Hotel
17,Kapaa,US,81.48,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
18,Pangai,TO,77.88,overcast clouds,-19.8000,-174.3500,Ha'apai Beach Resort


In [10]:
hotel_df.count()

City                   296
Country                296
Max Temp               296
Current Description    296
Lat                    296
Lng                    296
Hotel Name             296
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hithadhoo,MV,83.88,overcast clouds,-0.6000,73.0833,Scoop Guest House
1,Auki,SB,79.41,overcast clouds,-8.7676,160.7034,Auki Motel
6,Rikitea,PF,74.53,broken clouds,-23.1203,-134.9692,Pension Maro'i
7,Ca Mau,VN,80.55,overcast clouds,9.1769,105.1500,Khách Sạn Mường Thanh Luxury Cà Mau
8,Manado,ID,81.37,few clouds,1.4870,124.8455,Aryaduta Hotel
9,San Patricio,US,82.87,broken clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"
10,Jenison,US,70.70,overcast clouds,42.9072,-85.7920,"Holiday Inn Express Grand Rapids SW, an IHG Hotel"
12,Lang Son,VN,75.00,moderate rain,21.8333,106.7333,Khai Nga Hotel
17,Kapaa,US,81.48,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
18,Pangai,TO,77.88,overcast clouds,-19.8000,-174.3500,Ha'apai Beach Resort


In [12]:
# df['Tenant'].replace('', np.nan, inplace=True)
clean_hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
#df.dropna(subset=['Tenant'], inplace=True)
clean_hotel_df.dropna(subset=["Hotel Name"], inplace=True)
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hithadhoo,MV,83.88,overcast clouds,-0.6000,73.0833,Scoop Guest House
1,Auki,SB,79.41,overcast clouds,-8.7676,160.7034,Auki Motel
6,Rikitea,PF,74.53,broken clouds,-23.1203,-134.9692,Pension Maro'i
7,Ca Mau,VN,80.55,overcast clouds,9.1769,105.1500,Khách Sạn Mường Thanh Luxury Cà Mau
8,Manado,ID,81.37,few clouds,1.4870,124.8455,Aryaduta Hotel


In [13]:
# 8a. Create the output File (CSV)
output_data_file = os.path.join("..", "Vacation_Search", "WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))